# Zarr Demo

The following notebook illustrates the makeup and usage of zarr files.

We start with a simple synthetic example and then move to a 'real-world' dataset based on commonly used climate model output from the [pangeo cmip6 cloud archive](https://pangeo-data.github.io/pangeo-cmip6-cloud/accessing_data.html#opening-a-single-zarr-data-store)..

## Toy Example

In [7]:
import zarr
# create a 10x10 element array that is chunked along the first dimension
toystore = zarr.open('test.zarr', mode='w')
toystore

<zarr.hierarchy.Group '/'>

In [8]:
z = toystore.zeros('test_array', shape=(10, 10), chunks=(1, 10), dtype='i4')
z

<zarr.core.Array '/test_array' (10, 10) int32>

In [13]:
# populate the array with data
z[:] = 42

Now lets investigate the content of the created `test.zarr` folder.

In [14]:
!ls -la test.zarr/

total 16
drwxr-xr-x 3 jovyan root 4096 Mar  5 17:20 .
drwxr-xr-x 5 jovyan root 4096 Mar  5 17:22 ..
-rw-r--r-- 1 jovyan root   24 Mar  5 17:20 .zgroup
drwxr-xr-x 2 jovyan root 4096 Mar  5 17:23 test_array


We only added one array (`test_array`), which created a subfoler with the following content:

In [15]:
!ls -la test.zarr/test_array/

total 52
drwxr-xr-x 2 jovyan root 4096 Mar  5 17:23 .
drwxr-xr-x 3 jovyan root 4096 Mar  5 17:20 ..
-rw-r--r-- 1 jovyan root  334 Mar  5 17:20 .zarray
-rw-r--r-- 1 jovyan root   56 Mar  5 17:23 0.0
-rw-r--r-- 1 jovyan root   56 Mar  5 17:23 1.0
-rw-r--r-- 1 jovyan root   56 Mar  5 17:23 2.0
-rw-r--r-- 1 jovyan root   56 Mar  5 17:23 3.0
-rw-r--r-- 1 jovyan root   56 Mar  5 17:23 4.0
-rw-r--r-- 1 jovyan root   56 Mar  5 17:23 5.0
-rw-r--r-- 1 jovyan root   56 Mar  5 17:23 6.0
-rw-r--r-- 1 jovyan root   56 Mar  5 17:23 7.0
-rw-r--r-- 1 jovyan root   56 Mar  5 17:23 8.0
-rw-r--r-- 1 jovyan root   56 Mar  5 17:23 9.0


The actual data is written as one file per chunk, resulting in 10 files (`xx.0`), where `xx` indicates the chunknumber along the first dimension. The additional `.zarray` file contains information about the array like the number of elements along each dimension (`shape`), the size of chunks (`chunks`), and other parameters like compression.

In [28]:
import json
f = open('test.zarr/test_array/.zarray', 'r')
json.load(f)

{'chunks': [1, 10],
 'compressor': {'blocksize': 0,
  'clevel': 5,
  'cname': 'lz4',
  'id': 'blosc',
  'shuffle': 1},
 'dtype': '<i4',
 'fill_value': 0,
 'filters': None,
 'order': 'C',
 'shape': [10, 10],
 'zarr_format': 2}

## Real World Example

In [29]:
import gcsfs

# a CMIP6 zarr store
store = 'gs://cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/'

# Connect to Google Cloud Storage
fs = gcsfs.GCSFileSystem(token='anon', access='read_only')

# create a MutableMapping from a store URL
mapper = fs.get_mapper(store)

Using zarr a dataset not a single file, but instead is a folder with subfolders and files within.

In [30]:
# list the content of the zarr store
lst = fs.ls(store, detail=False)
lst

['cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/.zattrs',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/.zgroup',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/.zmetadata',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/lat',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/lat_bnds',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/lev',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/lev_bnds',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/lon',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/lon_bnds',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/thetao',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/the

This full structure is called a 'zarr group', representing several data arrays ('zarr arrays').
Each array represent n-dimensional arrays which are chunked. Lets investigate an example array `thetao`.

In [31]:
fs.ls('cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/thetao', detail=False)

['cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/thetao/.zarray',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/thetao/.zattrs',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/thetao/0.0.0.0',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/thetao/1.0.0.0',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/thetao/10.0.0.0',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/thetao/100.0.0.0',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/thetao/101.0.0.0',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/thetao/102.0.0.0',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/thetao/103.0.0.0',
 'cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v2019072

Data is stored for each chunk enumerated (xx.xx.xx.xx) along the dimensions of the array. We can guess that the data has 4 dimensions and is chunked along the first one (time in this case). All this information and other info like e.g. the compression are contained in the `.zarray` file.

In [32]:
def print_content(url):
    with fs.open(url, 'rb') as f:
        print(f.read().decode('utf-8'))

In [33]:
print_content('cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/thetao/.zarray')

{
    "chunks": [
        12,
        35,
        180,
        360
    ],
    "compressor": {
        "blocksize": 0,
        "clevel": 5,
        "cname": "lz4",
        "id": "blosc",
        "shuffle": 1
    },
    "dtype": "<f4",
    "fill_value": 1.0000000200408773e+20,
    "filters": null,
    "order": "C",
    "shape": [
        1980,
        35,
        180,
        360
    ],
    "zarr_format": 2
}


The `.zattrs` file contains metadata specific to the array (for instance variable units).

In [34]:
print_content('cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/thetao/.zattrs')

{
    "_ARRAY_DIMENSIONS": [
        "time",
        "lev",
        "lat",
        "lon"
    ],
    "cell_measures": "area: areacello volume: volcello",
    "cell_methods": "area: mean where sea time: mean",
    "comment": "Model data on the 1x1 grid includes values in all cells for which any ocean exists on the native grid. For mapping purposes, we recommend using a land mask such as World Ocean Atlas to cover these areas of partial land.  For calculating approximate integrals, we recommend multiplying by cell volume (volcello).",
    "interp_method": "conserve_order1",
    "long_name": "Sea Water Potential Temperature",
    "original_name": "thetao",
    "standard_name": "sea_water_potential_temperature",
    "units": "degC"
}


Going back to the group level, each group also contains a metadata file `.zattrs`

In [35]:
print_content('cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/.zattrs')

{"Conventions": "CF-1.7 CMIP-6.0 UGRID-1.0", "activity_id": "CMIP", "branch_method": "standard", "branch_time_in_child": 0.0, "branch_time_in_parent": 36500.0, "comment": "<null ref>", "contact": "gfdl.climate.model.info@noaa.gov", "coordinates": "time_bnds lon_bnds lat_bnds lev_bnds", "creation_date": "2019-08-03T00:22:55Z", "data_specs_version": "01.00.27", "experiment": "all-forcing simulation of the recent past", "experiment_id": "historical", "external_variables": "areacello volcello", "forcing_index": 1, "frequency": "mon", "further_info_url": "https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.GFDL-ESM4.historical.none.r1i1p1f1", "grid": "ocean data regridded from tripolar - nominal 0.5 deg latitude/longitude to 180,360; interpolation method: conserve_order1", "grid_label": "gr", "history": "File was processed by fremetar (GFDL analog of CMOR). TripleID: [exper_id_MFLg3OOf97,realiz_id_6UiFuoEKMa,run_id_PhuSv75why]", "initialization_index": 1, "institution": "National Oceanic and Atm

`.zgroup` must only contain the zarr storage version used [see here](https://zarr.readthedocs.io/en/stable/spec/v2.html?highlight=.zgroup#groups)

**@rabernat**: Is this correct?

In [36]:
print_content('cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/.zgroup')

{
    "zarr_format": 2
}


### Consolidated metadata
Zarr offers the functionality to [consolidate](https://zarr.readthedocs.io/en/stable/api/convenience.html?highlight=consolidate#zarr.convenience.consolidate_metadata) the metadata of a group to enable even faster lazy access (more on that in a bit). The information is stored in `.zmetadata`.

In [37]:
print_content('cmip6/CMIP6/CMIP/NOAA-GFDL/GFDL-ESM4/historical/r1i1p1f1/Omon/thetao/gr/v20190726/.zmetadata')

{
    "metadata": {
        ".zattrs": {
            "Conventions": "CF-1.7 CMIP-6.0 UGRID-1.0",
            "activity_id": "CMIP",
            "branch_method": "standard",
            "branch_time_in_child": 0.0,
            "branch_time_in_parent": 36500.0,
            "comment": "<null ref>",
            "contact": "gfdl.climate.model.info@noaa.gov",
            "coordinates": "time_bnds lon_bnds lat_bnds lev_bnds",
            "creation_date": "2019-08-03T00:22:55Z",
            "data_specs_version": "01.00.27",
            "experiment": "all-forcing simulation of the recent past",
            "experiment_id": "historical",
            "external_variables": "areacello volcello",
            "forcing_index": 1,
            "frequency": "mon",
            "further_info_url": "https://furtherinfo.es-doc.org/CMIP6.NOAA-GFDL.GFDL-ESM4.historical.none.r1i1p1f1",
            "grid": "ocean data regridded from tripolar - nominal 0.5 deg latitude/longitude to 180,360; interpolation method: 

## Accesing zarr files

### Directly with Zarr

In [38]:
import zarr

zstore = zarr.open_group(mapper)

zstore.info

Name,/
Type,zarr.hierarchy.Group
Read-only,False
Store type,fsspec.mapping.FSMap
No. members,9
No. arrays,9
No. groups,0
Arrays,"lat, lat_bnds, lev, lev_bnds, lon, lon_bnds, thetao, time, time_bnds"


Lets access the `thetao` array

In [39]:
zstore.thetao

<zarr.core.Array '/thetao' (1980, 35, 180, 360) float32>

@rabernat: I am actually not entirely sure how to access the zarr data from here? Let me know if this is at all useful.

Lets inspect the first chunk:

In [40]:
data = zstore.thetao.get_basic_selection(1)

In [41]:
import matplotlib.pyplot as plt

In [ ]:
plt.pcolor(data[0,:,:], vmax=40)
plt.colorbar()

### Opening zarr with xarray

In [ ]:
import xarray as xr
ds = xr.open_zarr(mapper, consolidated=True)
ds

Note that the file opened virtually instantaneous. This is due to the fact that we are working with a 'consolidated' zarr store, hence only one file has to be read to 'understand' the shape and metadata of the group (dataset in xarray lingo). 
None of the actual data is loaded at this point. Except for the dimension variables, all data is lazily loaded as dask arrays

In [ ]:
ds.thetao

xarray uses dask under the hood to load data lazily and parallelize computations across the chunks of each zarr array (Dataarray in xarray lingo)

In [ ]:
# Simple computation
mean_thetao = ds.thetao.isel(lev=0, time=slice(0,120)).mean(['lon', 'lat']) #average surface temperature over 10 years
mean_thetao

Again this did not yet trigger any computation, but dask builds a graph that shows how each chunk will be processed in parallel.

**@rabernat**: I wasnt able to cull this dask graph for a smaller graph visualization. do you know a quick way to do this? 

In [ ]:
mean_thetao.data.visualize()

Since each chunk contains 12 months worth of data, the computation over 10 years creates 10 branches in the task graph.

Plotting the resulting values finally triggers the computation:

In [ ]:
mean_thetao.plot()